In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output

In [2]:
df=pd.read_csv("tweets.csv")
df

,name,content,country,date_time,id,language,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/1/2017 19:52,8.200000e+17,en,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/1/2017 8:38,8.190000e+17,en,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/1/2017 2:52,8.190000e+17,en,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/1/2017 2:44,8.190000e+17,en,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/1/2017 5:22,8.190000e+17,en,17620,4655
...,...,...,...,...,...,...,...,...
42751,ddlovato,Life couldn't be better right now. 😊,NaN,6/1/2015 23:10,5.530000e+17,en,32799,23796
42752,ddlovato,First Monday back in action. I'd say 21.6 mile...,NaN,6/1/2015 2:17,5.520000e+17,en,21709,12511
42753,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",NaN,5/1/2015 3:42,5.520000e+17,en,25269,15583
42754,ddlovato,❄️ http://t.co/sHCFdPpGPa,NaN,5/1/2015 0:06,5.520000e+17,und,15985,10456


In [3]:
df["name"]= pd.Series(df["name"]).str.lower()
df

,name,content,country,date_time,id,language,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/1/2017 19:52,8.200000e+17,en,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/1/2017 8:38,8.190000e+17,en,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/1/2017 2:52,8.190000e+17,en,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/1/2017 2:44,8.190000e+17,en,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/1/2017 5:22,8.190000e+17,en,17620,4655
...,...,...,...,...,...,...,...,...
42751,ddlovato,Life couldn't be better right now. 😊,NaN,6/1/2015 23:10,5.530000e+17,en,32799,23796
42752,ddlovato,First Monday back in action. I'd say 21.6 mile...,NaN,6/1/2015 2:17,5.520000e+17,en,21709,12511
42753,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",NaN,5/1/2015 3:42,5.520000e+17,en,25269,15583
42754,ddlovato,❄️ http://t.co/sHCFdPpGPa,NaN,5/1/2015 0:06,5.520000e+17,und,15985,10456


In [4]:

df["date_time"] = pd.to_datetime(df["date_time"],dayfirst=True)
df = (
df.groupby([df["date_time"].dt.date, "name"])[
["number_of_likes", "number_of_shares"]
]
.mean()
.astype(int)
)
df = df.reset_index()
df


,date_time,name,number_of_likes,number_of_shares
0,2010-03-29,jtimberlake,24,61
1,2010-03-30,jtimberlake,9,23
2,2010-03-31,jtimberlake,16,31
3,2010-04-02,jtimberlake,59,176
4,2010-04-04,jtimberlake,29,57
...,...,...,...,...
12839,2017-01-21,jtimberlake,6461,1361
12840,2017-01-21,theellenshow,23909,4022
12841,2017-01-22,ladygaga,28210,9847
12842,2017-01-23,jtimberlake,13793,2381


In [ ]:
stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=stylesheets)

fig = px.bar(data_frame=df, x="name", y="number_of_likes",color="name", log_y=True)
app.layout = html.Div([
html.Div(html.H1("Twitter Likes Analysis of Famous People",
style={"textAlign": "center"}),
className="row"),    
html.Div(
dcc.Dropdown(id="people dropdown", multi=True,
options=[{"label": x, "value": x}
for x in sorted(df["name"].unique())],

style={"color": "green"}),
className="row"),

html.Div(dcc.Graph(id="line-chart", figure=fig), className="row")

])
@app.callback(Output(component_id="line-chart", component_property="figure"),[Input(component_id="people dropdown", component_property="value")])
def update_graph(chosen_value):
    print(f"Values chosen by user: {chosen_value}")
    if len(chosen_value) == 0:
        return {}
    else:
        df_filtered = df[df["name"].isin(chosen_value)]
        fig = px.line(
        data_frame=df_filtered,
        x="date_time",
        y="number_of_likes",
        color="name",
        log_y=True,
        labels={"number_of_likes": "Likes","date_time": "Date","name": "Celebrity",})

    return fig


# Run the Dash app
if __name__ == "__main__":
    app.run_server(mode="jupyterlab")
